# Installation & Setup

In [1]:
%%capture
!pip install nltk
!pip install transformers
!pip install datasets
!pip install srt
!pip install gdown
!apt install ffmpeg
!pip install deepmultilingualpunctuation
!pip install silero-vad
!pip install spacy
!pip install pytextrank
!pip install pydub

!pip install git+https://github.com/m-bain/whisperX.git

In [2]:
import os
import numpy as np
import pandas as pd
from google.colab import files
import tarfile
import gdown
import nltk
from nltk.tokenize import sent_tokenize
import spacy
import srt
import re
from functools import reduce
from pydub import AudioSegment

from datasets import load_dataset
import torch
import torchaudio
import torch.nn.functional as F
from transformers import \
LongformerTokenizer, LongformerModel, LongformerForSequenceClassification, LongformerConfig, \
RobertaTokenizer, RobertaForTokenClassification, Trainer, TrainingArguments, \
LEDTokenizer, LEDForConditionalGeneration

# Text
import pytextrank

# Audio
import whisperx
import silero_vad
from silero_vad import load_silero_vad, read_audio, get_speech_timestamps

# Video

/usr/local/lib/python3.10/dist-packages


/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/pyannote/audio/pipelines/speaker_verification.py:43: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  backend = torchaudio.get_audio_backend()
/usr/local/lib/python3.10/dist-packages/pyannote/audio/pipelines/speaker_verification.py:45: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import (
/usr/local/lib/python3.10/dist-packages/pyannote/audio/pipelines/speaker_

In [3]:
%%capture
# NLTK
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
# Notebook config
os.environ["WANDB_DISABLED"] = "true"

# Variables

In [5]:
video_input = "dataset/teamwork in the classroom.mov"

full_base = os.path.dirname(video_input)
path_dataset = full_base
filename = os.path.basename(video_input)
filename_without_extension = os.path.splitext(filename)[0]

filename_video_input = filename
filename_subtitles_output = filename_without_extension + ".srt"
filename_audio_output = filename_without_extension + ".wav"

subtitles_output = os.path.join(full_base, filename_subtitles_output)
audio_output = os.path.join(full_base, filename_audio_output)

video = ''
audio = ''
subtitles = ''
sentences = ''

# Datasets

In [6]:
# Google Drive Dataset Location
folder_id = '1k7DLJPl1xz9lpU4l3dZYtPe1XawhrXeC' # taken from drive.google.com/drive/u/1/folders/1k7D...(this part)
gdown.download_folder(id=folder_id, quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 1vuIW3CVm2p_Ig-_srJ5sIwUxzTGqdGHp assessing students without exams.mov
Processing file 1OP3zzSmpKJ0RDPasl9AGQD2yNeCEXtoR flipped learning basics.mov
Processing file 1wslcvTNd88FQMXJgvGbKR3sjwXORR6xt teamwork in the classroom.mov
Processing file 1XdQdQ-sEqgICmiSpIQvzloD-Cvc9WHFk teamwork in the classroom.srt
Processing file 1lb2rCvjouElVhqEHAyojUOvb_ECgUqjF teamwork in the classroom.wav


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1vuIW3CVm2p_Ig-_srJ5sIwUxzTGqdGHp
From (redirected): https://drive.google.com/uc?id=1vuIW3CVm2p_Ig-_srJ5sIwUxzTGqdGHp&confirm=t&uuid=3c2572a6-6b5e-4b93-ba88-dcb21578684c
To: /content/dataset/assessing students without exams.mov
100%|██████████| 875M/875M [00:13<00:00, 65.4MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1OP3zzSmpKJ0RDPasl9AGQD2yNeCEXtoR
From (redirected): https://drive.google.com/uc?id=1OP3zzSmpKJ0RDPasl9AGQD2yNeCEXtoR&confirm=t&uuid=08a742fc-5334-40b2-8af3-3bbb1b602ad3
To: /content/dataset/flipped learning basics.mov
100%|██████████| 399M/399M [00:08<00:00, 46.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1wslcvTNd88FQMXJgvGbKR3sjwXORR6xt
From (redirected): https://drive.google.com/uc?id=1wslcvTNd88FQMXJgvGbKR3sjwXORR6xt&confirm=t&uuid=7691330b-4dcd-42fe-87b3-

['/content/dataset/assessing students without exams.mov',
 '/content/dataset/flipped learning basics.mov',
 '/content/dataset/teamwork in the classroom.mov',
 '/content/dataset/teamwork in the classroom.srt',
 '/content/dataset/teamwork in the classroom.wav']

In [7]:
# Simple Test Dataset:
paragrah_simple = "Renewable energy is crucial for reducing carbon emissions. Solar power, in particular, is sustainable and abundant. Interestingly, solar panels were first invented in 1954. With continued advancements, solar energy is becoming more accessible in everyday life."
paragraph_simple_unpunct = "Renewable energy is crucial for reducing carbon emissions  Solar power, in particular, is sustainable and abundant Interestingly, solar panels were first invented in 1954 With continued advancements, solar energy is becoming more accessible in everyday life"

# Other: CNN/Daily Mail
# dataset_news = load_dataset("cnn_dailymail", "3.0.0")
# paragraph_news = dataset_news['train']['article'][0]
# summary_news = dataset_news['train']['highlights'][0]

SRT  
each **`subtitle`** in the subtitles array has the following properties:

1. **`index`**
   - The sequential number of the subtitle within the SRT file.
   - `1`, `2`, `3`, etc. (Integer)
2. **`start`**
   - The time (in milliseconds) when the subtitle should appear on the screen.
   - `00:00:05,000` (String representing HH:MM:SS,SSS)
3. **`end`**
   - The time (in milliseconds) when the subtitle should disappear from the screen.
   - `00:00:10,000` (String representing HH:MM:SS,SSS)
4. **`content`**
   - The actual text of the subtitle that will be displayed.
   - "Hello, world!" (String)
5. **`proprietary`**
   - This field holds any additional data or formatting specific to the SRT file or software used to create it. Often empty and can usually be ignored.
   - `''` (Empty string, or sometimes contains specific formatting codes)

# Preprocessing

## Audio - Load

In [ ]:
%%capture
# Extract audio (wav) from video
!ffmpeg -y -i "$video_input" -vn -acodec pcm_s16le -ar 44100 -ac 2 "$audio_output"

## Audio - SRT Generation

##### Time Taken: ~4min

In [37]:
def seconds_to_srt_timestamp(seconds):
    """
    Extract hours, minutes, seconds, and milliseconds
    from a given number of seconds.
    """

    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    milliseconds = int((seconds - int(seconds)) * 1000)

    # Format as HH:MM:SS,MS
    return f"{hours:02}:{minutes:02}:{int(seconds):02},{milliseconds:03}"

In [38]:
# Select device (GPU if available, otherwise CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
language="en"
compute_type="int8"

# Model WhisperX
model = whisperx.load_model("base", device=device, language=language, compute_type=compute_type) # Choose "base" or "large" model

# Transcribe audio
aligned_segments = model.transcribe(audio_output)

# Align with forced alignment
alignment_model, metadata = whisperx.load_align_model(language_code=aligned_segments["language"], device=device)
aligned_segments = whisperx.align(aligned_segments["segments"], alignment_model, metadata, audio_output, device)

# Generate SRT file with aligned sentences
with open(subtitles_output, "w") as f:
    for i, segment in enumerate(aligned_segments["segments"], 1):
        # Get start and end times in SRT format
        start_time = seconds_to_srt_timestamp(segment["start"])
        end_time = seconds_to_srt_timestamp(segment["end"])

        # Write SRT entry
        f.write(f"{i}\n{start_time} --> {end_time}\n{segment['text']}\n\n")

print(f"SRT file generated: {subtitles_output}")

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.0+cu121. Bad things might happen unless you revert torch to 1.x.
SRT file generated: dataset/teamwork in the classroom.srt


## Text - Sentence Segmentation

In [39]:
# Subtitles:
with open(subtitles_output, "r", encoding="utf-8") as f:
    subtitles = list(srt.parse(f.read()))

In [ ]:
sentences = []
for i, segment in enumerate(subtitles["segments"], 1):
    sentences.append({
        'base_idx': i-1,
        'start_time': segment["start"],
        'end_time': segment["start"],
        'sentence': segment['text']
    })

df_sentences = pd.DataFrame(sentences)
sentences = df_sentences['sentence'].tolist()

display(df_sentences)

,base_idx,start_time,end_time,sentence
0,0,00:01.274,00:08.685,"Hello, this is Lino Cordia and today I want t..."
1,1,00:11.058,00:13.319,So why is teamwork in the classroom so import...
2,2,00:13.359,00:17.480,"Well, for our students, it allows them to deve..."
3,3,00:17.700,00:20.061,"Communication skills, leadership skills, et ce..."
4,4,00:20.621,00:23.862,"Also, when you're working with a team, you get..."
...,...,...,...,...
69,69,05:46.624,05:48.245,"And yeah, please reach out."
70,70,05:48.385,05:49.406,I would love to hear from you.
71,71,05:49.526,05:50.266,Thank you so much.
72,72,05:50.547,05:51.487,I am Lino Cordia.


## Text - Paragraph
combination of all subtitle parts.  

WhisperAI enhances transcription with basic punctuation.

In [ ]:
# Assuming aligned_segments is your WhisperX alignment output
paragraph = reduce(lambda acc, seg: acc + seg['text'].strip() + ' ', aligned_segments['segments'], '')

# Print the paragraph
print(paragraph)

Hello, this is Lino Cordia and today I want to talk to you about a very important topic, challenging topic, teamwork in the classroom. So why is teamwork in the classroom so important? Well, for our students, it allows them to develop a bunch of new skills, right? Communication skills, leadership skills, et cetera. Also, when you're working with a team, you get different perspectives. Ideally, you are part of a team that has people with different genders, people with different age groups, people with different academic backgrounds, right? So when you're talking to them, you get all these fresh perspectives that inform your the task that you're trying to solve. Also teams will motivate you, they will support you, you will feel empowered by them ideally, right? This is like the things that should happen. And also this is how the world works, right? Pretty much everything we do, we need to do it as a team. Now here's the thing about teamwork in the classroom. Students hate it. But why do 

## Text - Paragraph Summarized

##### Time Taken: ~1min

In [ ]:
# Model: Longformer Encoder-Decoder
model_name = "allenai/led-base-16384"
tokenizer = LEDTokenizer.from_pretrained(model_name)
model = LEDForConditionalGeneration.from_pretrained(model_name)
text = paragraph

# Tokenization
inputs = tokenizer(text, return_tensors="pt", max_length=4096, truncation=True)

# Summary Generation
summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
paragraph_summarized = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Input ids are automatically padded from 1141 to 2048 to be a multiple of `config.attention_window`: 1024


# Text

In [ ]:
# Simple Metrics
original_length = len(paragraph)
summary_length = len(paragraph_summarized)

print(f"original length: {original_length}")
print(f"summary length: {summary_length}")

compression_ratio = (original_length - summary_length) / original_length
print(f"auto-summary compression ratio: {compression_ratio:.2f}")

original length: 5209
summary length: 708
auto-summary compression ratio: 0.86


## Metric 1: Sentence- Summarized Paragraph Relevancy (Cosine Similarity)

In [ ]:
%%capture
# config
attention_window = 256
config = LongformerConfig.from_pretrained('allenai/longformer-base-4096', attention_window=attention_window)

# model: Longformer
model_lf = LongformerModel.from_pretrained('allenai/longformer-base-4096', config=config)
tokenizer_lf = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096', model_max_length=attention_window)

In [ ]:
# 2: Tokenization
paragraph_tokens = tokenizer_lf(paragraph_summarized, return_tensors='pt')
# sentence_tokens = [tokenizer_lf(sentence, return_tensors='pt') for sentence in sentences]

sentence_tokens = tokenizer_lf(sentences, padding=True, truncation=True, return_tensors='pt')

Embedding Explanation  
The [CLS] (classification) token is often used in transformer models to represent the overall meaning or summary of the input sequence. By extracting its embedding, you're essentially obtaining a representation that captures the main point or essence of the paragraph.

##### Time Taken: ~2min

In [ ]:
# 3: Embedding
with torch.no_grad():  # Disable gradient computation for efficiency
    paragraph_embedding = model_lf(**paragraph_tokens).last_hidden_state[:, 0, :]  # Get the [CLS] token embedding

    # Process batched sentence tokens
    sentence_embeddings = model_lf(**sentence_tokens).last_hidden_state[:, 0, :]

Input ids are automatically padded from 150 to 256 to be a multiple of `config.attention_window`: 256
Input ids are automatically padded from 40 to 256 to be a multiple of `config.attention_window`: 256


In [ ]:
# 4: Relevance scores
relevance_scores = [torch.cosine_similarity(paragraph_embedding, sentence_embedding).item() for sentence_embedding in sentence_embeddings]

# Normalization: min-max normalization
min_score = min(relevance_scores)
max_score = max(relevance_scores)
normalized_scores = [(score - min_score) / (max_score - min_score) for score in relevance_scores]

# round
normalized_scores = [np.format_float_positional(score, precision=2, unique=False, fractional=False, trim='k') for score in normalized_scores]

In [ ]:
# 5: Display Results
df_sentences.insert(0, "metric_1_score", normalized_scores)

display(df_sentences)

,metric_1_score,base_idx,start_time,end_time,sentence
0,1.0,0,00:01.274,00:08.685,"Hello, this is Lino Cordia and today I want t..."
1,0.45,1,00:11.058,00:13.319,So why is teamwork in the classroom so import...
2,0.57,2,00:13.359,00:17.480,"Well, for our students, it allows them to deve..."
3,0.47,3,00:17.700,00:20.061,"Communication skills, leadership skills, et ce..."
4,0.53,4,00:20.621,00:23.862,"Also, when you're working with a team, you get..."
...,...,...,...,...,...
69,0.39,69,05:46.624,05:48.245,"And yeah, please reach out."
70,0.33,70,05:48.385,05:49.406,I would love to hear from you.
71,0.51,71,05:49.526,05:50.266,Thank you so much.
72,0.25,72,05:50.547,05:51.487,I am Lino Cordia.


## Metric 2: Intra-sentence relevancy
Score by if current sentence is needded by adjacent sentences.

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Store predictions for each sentence
predictions = []

# Iterate through sentence pairs
for i in range(len(sentences) - 1):
    sentence1 = sentences[i]
    sentence2 = sentences[i + 1]

    # Tokenize and prepare input
    inputs = tokenizer(sentence1, sentence2, return_tensors='pt', truncation=True, padding=True, add_special_tokens=True)

    # Get model prediction
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits).item()

    # Store prediction
    predictions.append(prediction)

# Handle last sentence (no next sentence)
predictions.append(0)  # Assume last sentence doesn't need a next sentence

In [ ]:
# Add predictions to DataFrame
df_sentences = df_sentences.assign(**{"Previous Sentence Needed": predictions})

display(df_sentences)

## Metric 3: Intelligent Sentence-Paragraph Relevancy

##### Time Taken: 13min - 26min

In [ ]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096")

# Ensure the model is in evaluation mode
model.eval()

# Example usage
body_paragraph = paragraph

relevance_scores = []

for sentence in sentences:
    # Prepare the input for Longformer
    inputs = tokenizer(
        body_paragraph,
        sentence,
        return_tensors='pt',
        max_length=4096,
        truncation=True,
        padding='max_length'  # Pad to max length to avoid issues with model input size
    )

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Assuming binary classification (relevant/not relevant)
    relevance_score = torch.softmax(outputs.logits, dim=1)[0][1].item()  # Probability of being relevant
    relevance_scores.append((sentence, relevance_score))

# Sort sentences based on relevance scores
sorted_sentences = sorted(relevance_scores, key=lambda x: x[1], reverse=True)
ranked_sentences = [sentence for sentence, score in sorted_sentences]

In [ ]:
relevance_scores[0]

In [ ]:
sentence_indices = list(range(len(relevance_scores)))
scores = [score for sentence, score in relevance_scores]
sentences_text = [sentence for sentence, score in relevance_scores]

df_relevance = pd.DataFrame({'Sentence Index': sentence_indices, 'Score': scores, 'Sentence': sentences_text})
df_relevance

## Metric 4: Keyword extraction and Ranking
using TextRank

In [ ]:
# Load a spaCy model
nlp = spacy.load("en_core_web_sm")

# Add the pytextrank pipeline component to spaCy
nlp.add_pipe("textrank")

phrase_data = []

# Process the text
doc = nlp(paragraph)

for phrase in doc._.phrases:
  phrase_data.append([phrase.text, phrase.rank, phrase.count])

df_phrases = pd.DataFrame(phrase_data, columns=['Phrase', 'Rank', 'Count'])
df_phrases.sort_values(by=['Rank'], ascending=False, inplace=True)

display(df_phrases)

# Audio

## Metric 5: Silence Detection
* From the Paragraph boundaries, get the time in aduio that we care about
* For each time in audio we care about, analyze if they are low volume

OR
* analyze all potential sentence boundaries first
* match with end of sentences

In [ ]:
# 0: Load audio, extract timestamps

SAMPLING_RATE = 16000 # 16 kHz

model = load_silero_vad()
wav = read_audio(audio_output)
speech_timestamps = get_speech_timestamps(wav, model)

# Check the shape of the wav tensor
print(f"Audio shape: {wav.shape}")
print(f"Audio length (seconds): {len(wav) / SAMPLING_RATE:.2f}")

Audio shape: torch.Size([5657259])
Audio length (seconds): 353.58


In [ ]:
# Speech Intervals
speech_intervals = []
for i in range(0, len(speech_timestamps)-1):
    speech_intervals.append((speech_timestamps[i]['start'] / SAMPLING_RATE, speech_timestamps[i]['end'] / SAMPLING_RATE))

# Silence Intervals
silence_intervals = []
for i in range(1, len(speech_timestamps)):
    silence_start = speech_timestamps[i-1]['end']  # End of previous speech segment
    silence_end = speech_timestamps[i]['start']     # Start of current speech segment
    silence_intervals.append((silence_start / SAMPLING_RATE, silence_end / SAMPLING_RATE))

In [ ]:
print(speech_timestamps[0:3])
print(speech_intervals[0:3])
print(silence_intervals[0:3])

[{'start': 20000, 'end': 113120}, {'start': 116768, 'end': 140256}, {'start': 176672, 'end': 219104}]
[(1.25, 7.07), (7.298, 8.766), (11.042, 13.694)]
[(7.07, 7.298), (8.766, 11.042), (13.694, 14.05)]


# Video

# PostProcessing

In [ ]:
df_sentences.drop('metric_final', axis=1, inplace=True)
df_sentences.insert(0, 'metric_final', 1)

# Metric 1 Apply
df_sentences['metric_final'] = 1 * df_sentences['metric_final'] * df_sentences['metric_1_score'].astype(float)

display(df_sentences)

,metric_final,metric_1_score,base_idx,start_time,end_time,sentence
0,1.00,1.0,0,00:01.274,00:08.685,"Hello, this is Lino Cordia and today I want t..."
1,0.45,0.45,1,00:11.058,00:13.319,So why is teamwork in the classroom so import...
2,0.57,0.57,2,00:13.359,00:17.480,"Well, for our students, it allows them to deve..."
3,0.47,0.47,3,00:17.700,00:20.061,"Communication skills, leadership skills, et ce..."
4,0.53,0.53,4,00:20.621,00:23.862,"Also, when you're working with a team, you get..."
...,...,...,...,...,...,...
69,0.39,0.39,69,05:46.624,05:48.245,"And yeah, please reach out."
70,0.33,0.33,70,05:48.385,05:49.406,I would love to hear from you.
71,0.51,0.51,71,05:49.526,05:50.266,Thank you so much.
72,0.25,0.25,72,05:50.547,05:51.487,I am Lino Cordia.


In [ ]:
# Load the audio file
audio = AudioSegment.from_file(audio_output)

output_temp = "trimmed.wav"

In [ ]:
trimmed_audio = audio_trim(audio, ts_to_ms("00:7.00"), ts_to_ms("00:11.00"))


# Export the trimmed audio
trimmed_audio.export(output_temp, format="wav")

<_io.BufferedRandom name='trimmed.wav'>

In [ ]:
# Download
files.download(output_temp)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from IPython.display import Audio

In [ ]:
def ts_to_ms(timestamp):
    # Format "MM:SS.mmm"
    minutes, seconds, milliseconds = map(int, re.split(r'[:.]', timestamp))
    # Convert to milliseconds
    return (minutes * 60 + seconds) * 1000 + milliseconds

# Example usage
timestamp = "05:48.385"
ms = ts_to_ms(timestamp)
print(f"Milliseconds: {ms}")

Milliseconds: 348385


In [ ]:
def audio_trim(audio, start_trim, end_trim):
    """
    Trims a single segment from an AudioSegment object.

    Args:
        audio: The AudioSegment object to trim.
        start_trim: The start time of the segment to remove (in milliseconds).
        end_trim: The end time of the segment to remove (in milliseconds).

    Returns:
        The AudioSegment object with the specified segment removed.
    """
    trimmed_audio = audio[:start_trim] + audio[end_trim:]
    return trimmed_audio